referencias recomendadas:
- https://www.youtube.com/watch?v=2pWv7GOvuf0&t=130s
- https://keon.io/deep-q-learning/
- https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0
- https://en.wikipedia.org/wiki/Q-learning

# Reinforcement Learning

![diagram.png](diagram.png)

- No hay supervisión, la realimentación está basada en una recompenza (**Reward**)
- La realimentación puede estar atrasada en el tiempo (Instant reward, time delayed reward, **discount factor**)
- **No** son secuencias I.I.D como en muchos modelos de aprendisaje supervisado (Secuencia correlacionada)
- Un **Agente** que toma acciones y los resultados dependen directamente de ellas

# Ejemplos:

- Mañobrar un Drone
- Jugar al Backgammon
- Administrar un portfolio de inversión
- Lograr que un robot camine
- Jugar juegos de Atari

¿Cual es la recompenza en cada caso? ¿Esta atrasada en el tiempo?

# Recompenza (Reward) $R_t$

- Es un escalar
- Indica que tan bien el agente lo esta haciendo en el tiempo t

# Objetivo: Tomar las acciones que maximizen la recompenza acumulada

- Las acciones pueden tener consecuencias a largo plazo
- La recompenza puede estar retrasada en el tiempo
- Puede ser importante rechazar recompenza inmediata para tener recompenzas mayores al largo plazo

Ejemplo:
- Una inversión financiera puede tardar meses en hacerse efectiva
- En ajedrez bloquear al oponente puede ser una estrategia mejor que comer una pieza

# Modelo

![RL-Modelo.png](RL-Modelo.png)

# Historia: es la secuencia de observaciones, Recompenzas y Acciones hasta el momento actual

$\huge H_t = O_1, R_1, A_1, ..., A_{t-1}, O_t, R_t$

# STATE: Es una función de la historia

$\huge S_t = f(H_t)$

Es lo que se usa para determinar que será lo proximo que ocurrira

El Agente tiene su propia representación del estado del entorno (environmient)

# Markov Desision Process (MDP)

$\huge P[S_{t+1}|S_t] = P[S_{t+1}|S_1, S_2, ..., S_t]$

Que consideramos como estado?

![rat.png](rat.png)

# El agente construye su propia representación:
- Historia completa
- Un red recurrente
- Combinadciones de los estados

# Componentes de un Agente en RL
Nuestra meta es construir el agente que pueda tomar las decisiones

- Policy: Como debe comportarse el Agente
- Value Function: Que tan bueno es cada estado
- Q Funcrion: Que tan bueno es cada estada con cada accion
- Modelo: Representación del entorno

# Policy

Deterministica:

$\huge a=\pi(s)$

Estocastica:

$\huge \pi(a|s) = P[A_t=a|S_t=s]$

Normalmente usamos la deterministica

![policy.png](policy.png)

# Value Function (Bellman Equation)
- Predicción de la recompenza futura partiendo del estado **s**
- Evalua que tan bueno es un estado
- Permite tomar decisiones
- **No** es lo mismo que la recompenza en un estado

![value_func.png](value_func.png)

$\huge v_{\pi}(s) = E_{\pi}[R_{t+1}+\gamma R_{t+2}+\gamma^2 R_{t+3}+...|S_t = s]$

El subindice $\pi$ indica que es para una policy particular

La esperanza es debido al caracter aleatorio. Por ejemplo, cuando tomo una decisión, el proximo estado puede ser aleatorio.

$\huge v_{\pi}(s) = E_{\pi}[R_{t+1}+\gamma v_{\pi}(s_{t+1})|S_t = s]$

# Action Value Function

$\huge q_{\pi}(s, a) = E_{\pi}[R_{t+1}+\gamma q_{\pi}(s_{t+1}, a_{t+1})|S_t = s, A_t=a]$

$\huge v_{\pi}(s) = \sum_a \pi(a|s)q_{\pi}(s,a)$

# Objetivo: encontrar la política $\pi$ que maximize la Value Function Q

Si puedo encontrar Q luego solo tengo que hacer:

$\huge \pi(s) = argmax_{a} q(s,a)$

# Q-Learning
Metodo iterativo:

## Tabla

### Promedio Ponderado

$\huge q(s_t, a_t) \leftarrow (1-\alpha)q(s_t, a_t) + \alpha (R_t +\gamma max_a[q(s_{t+1},a)])$

- Si el entorno (envornment) es totalmente deterministico entonces $\alpha=1$ es optimo
- Si $\alpha=0$ el agente no aprende
- $\gamma = 1$ considera las recompensas futuras tan importantes como las inmediatos
- $\gamma = 0$ Solo le interesa la recompenza inmediata

### Minimización del error

$\huge \hat{q}(s_t,a_t) = R +\gamma max_a[q(s_{t+1},a)]$

error:

$\huge e = \hat q(s_t,a) - q(s_t,a)$

Minimizar error cuadratico

$\huge q(s_t,a) = q(s_t,a)+\alpha e$

donde $\alpha$ es el learning rate 

## Red Neuronal

- Misma promedio ponderado
- Pero estima q(s,a) con una red

Algoritmo:
- Inicializo pesos de la red neuronal aleatoriamente
- Ingreso con el estado inicial $s_0$
- Me devuelve un listado de posibles acciones con sus pesos: $q(s_0, a)$
- Elijo la accion mas probable a_0 (Con un grado de aleatoriedad que va disminuyendo con cada episodio) y guardo el valor $maxQ_0$ (valor de Q para esa accion)
- Ejecuto la accion y tengo un nuevo estado $S_1$ con un posible reward $r_1$
- Ingreso a la RNN con ese nuevo estado, obtengo $q(s_1,a)$ y me quedo con el valor mayor ($maxQ_1$)
- Uso $maxQ_1$ para predecir el anterior $maxQ_0$: $r_1+\gamma maxQ_1$
- Es decir, le pido que haga un update de los pesos con entrada $s_0$ y salida $maxQ_0$ (Target, valor anotado)
- Repito esto hasta que termina el juego (suponiendo que termina)
- repito esto tantos episodios como sean necesarios

# Model Based - Model Free
Si es **model based**:

- Intenta predecir lo como se comportará el entorno en cada paso
- $P$ predice el próximo estado
- $R$ predice la próxima recompenza inmediata (Inmediate reward)

$\huge P_{SS^{\prime}}^a = P[S_{t+1} = s^{\prime}|S_t = s, A_t = a]$

$\huge R_s^a = E[R_{t+1}|S_t=s, A_t = a]$